In [ ]:
import pandas as pd
from load_data import full_kenpom_pipeline, read_unplayed_tournament, load_player_data
from simulate_tournament import simulate_n_tournaments

In [ ]:
year = 2024
kenpom_ratings_df = full_kenpom_pipeline(year)
matchups_dict = read_unplayed_tournament(year)
player = load_player_data(year, matchups_dict)
probs, sims = simulate_n_tournaments(matchups_dict, player, kenpom_ratings_df, N=10000)

# Convert the probabilities to a DataFrame for better readability
df = pd.DataFrame(probs.items(), columns=['Team', 'Probability'])
df = df.sort_values(by='Probability', ascending=False)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [6]:
# Sanity Checks

print(df['Probability'].sum()) # Check that the sum of probabilities is 1
print(len(sims)) # Check that the number of simulations matches the specified N

In [7]:
sims[4].players_bookkeeping['UConn']

In [9]:
target_players = {
    ("Purdue", "Zach Edey"): 1,
    ("Tennessee", "Dalton Knecht"): 1,
    ("Houston", "LJ Cryer"): 1,
    ("Clemson", "PJ Hall"): 2,
    ("Dayton", "DaRon Holmes II"): 2,
    ("Morehead State", "Riley Minix"): 3,
    ("Michigan State", "Tyson Walker"): 2,
    ("UNC", "RJ Davis"): 1,
    ("Florida", "Walter Clayton Jr."): 2,
    ("Florida", "Zyon Pullin"): 2,
    ("Clemson", "Joseph Girard III"): 2,
    ("Texas", "Max Abmas"): 2,
    ("Texas Tech", "Pop Isaacs"): 2,
    ("BYU", "Jaxson Robinson"): 2,
    ("Akron", "Enrique Freeman"): 3
}

global_sum_avg = 0
num_sims = len(sims)
print("Player Average Running Totals Over All Simulations:")

for (team, player), multiplier in target_players.items():
    total_sum = 0
    sim_count = 0
    for sim in sims:
        # Retrieve the player's running_total, if available in the simulation bookkeeping
        running_total = sim.players_bookkeeping.get(team, {}).get(player, {}).get("running_total")
        if running_total is not None:
            total_sum += running_total
            sim_count += 1
    if sim_count > 0:
        avg_total = total_sum / sim_count
        print(f"Team: {team:15s} | Player: {player:20s} | "
              f"Average Running Total: {avg_total:.2f} (Expected x{multiplier})")
        global_sum_avg += avg_total
    else:
        print(f"Team: {team:15s} | Player: {player:20s} | Total Not Found")

print("\nGlobal Sum of Averages for Target Players:", global_sum_avg)

In [10]:
global_totals = {}

for sim in sims:
    for team, players in sim.players_bookkeeping.items():
        for player, stats in players.items():
            key = (team, player)
            global_totals.setdefault(key, []).append(stats['running_total'])

global_players = []
for (team, player), totals in global_totals.items():
    avg_total = sum(totals) / len(totals)
    global_players.append((team, player, avg_total))

# Sort players by average running total in descending order
global_sorted = sorted(global_players, key=lambda x: x[2], reverse=True)

for team, player, avg_total in global_sorted:
    print(f"Team: {team:10s} | Player: {player:15s} | Average Running Total: {avg_total:.2f}")

In [ ]:
global_players = []
for team, players in sims[0].players_bookkeeping.items():
    for player, stats in players.items():
        global_players.append((team, player, stats['running_total']))

# Sort all players by running_total in ascending order
global_sorted = sorted(global_players, key=lambda x: x[2], reverse=True)

for team, player, running_total in global_sorted:
    print(f"Team: {team:10s} | Player: {player:15s} | Running Total: {running_total}")

In [14]:
from select_strategy import score_strategy

In [ ]:
score_strategy()